In [1]:

!pip install fastapi pyngrok uvicorn nest-asyncio python-multipart opencv-python ffmpeg-python
!pip install av torch torchvision tqdm pims


!git clone https://github.com/PeterL1n/RobustVideoMatting.git
%cd RobustVideoMatting


!wget https://github.com/PeterL1n/RobustVideoMatting/releases/download/v1.0.0/rvm_resnet50.pth -O rvm_resnet50.pth


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.8/87.8 kB 6.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.7/39.7 MB 34.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 96.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 78.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 51.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 97.3 MB/s eta 0:00:00
  C

In [2]:

from fractions import Fraction


file_path="/content/RobustVideoMatting/inference_utils.py"
with open(file_path, "r") as file:
    lines = file.readlines()


if "from fractions import Fraction" not in "".join(lines):
    for i, line in enumerate(lines):
        if line.strip().startswith("import") or line.strip().startswith("from"):
            continue
        else:
            lines.insert(i, "from fractions import Fraction\n")
            break


for i, line in enumerate(lines):
    if "self.container.add_stream('h264'" in line:
        lines[i] = "        self.stream = self.container.add_stream('h264', rate=Fraction(frame_rate).limit_denominator())\n"
        break

with open(file_path, "w") as f:
    f.writelines(lines)

print("Patched RVM inference_utils.py")


Patched RVM inference_utils.py


In [3]:
from pyngrok import conf, ngrok
import nest_asyncio
from fastapi import FastAPI, File, UploadFile, Query
from fastapi.responses import FileResponse, JSONResponse
import uvicorn
import os,shutil
import cv2
import numpy as np
import aiohttp
import asyncio
import sys
import subprocess
import time


conf.get_default().auth_token = "30MuVGxZbI6iVPrphbECYG5tUP7_7hmuPJqr3HATeSG9iBNMk"


app = FastAPI()
os.makedirs("temp", exist_ok=True)


def run_rvm(input_path, fg_path, alpha_path, checkpoint="rvm_resnet50.pth", device="cuda"):
    command = [
        "python", "-m", "inference",
        "--variant", "resnet50",
        "--checkpoint", checkpoint,
        "--device", device,
        "--input-source", input_path,
        "--output-type", "video",
        "--output-foreground", fg_path,
        "--output-alpha", alpha_path,
        "--seq-chunk", "12",
        "--downsample-ratio", "0.25",
        "--output-video-mbps", "6"
    ]

    print("Running RVM with command:", " ".join(command))
    result = subprocess.run(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)

    print("RVM stdout:\n", result.stdout)
    print("RVM stderr:\n", result.stderr)

    if not os.path.exists(fg_path) or not os.path.exists(alpha_path):
        raise Exception("Foreground or alpha video not generated. RVM may have failed.")


def composite_video(fg_path, alpha_path, bg_path, output_path="temp/composited_output.mp4"):
    import subprocess

    cap_fg = cv2.VideoCapture(fg_path)
    cap_alpha = cv2.VideoCapture(alpha_path)
    cap_bg = cv2.VideoCapture(bg_path)

    ret, first_fg = cap_fg.read()
    ret_alpha, first_alpha = cap_alpha.read()
    ret_bg, first_bg = cap_bg.read()



    height, width, _ = first_fg.shape
    cap_fg.set(cv2.CAP_PROP_POS_FRAMES, 0)
    cap_alpha.set(cv2.CAP_PROP_POS_FRAMES, 0)
    cap_bg.set(cv2.CAP_PROP_POS_FRAMES, 0)

    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    fps = cap_fg.get(cv2.CAP_PROP_FPS) or 10
    temp_output_path = "temp/temp_composite_raw.mp4"
    out = cv2.VideoWriter(temp_output_path, fourcc, fps, (width, height))

    frame_count = 0
    bg_frames = []

    while True:
        ret_bg, bg_frame = cap_bg.read()
        if not ret_bg:
            break
        bg_frame = cv2.resize(bg_frame, (width, height))
        bg_frames.append(bg_frame)

    if not bg_frames:
        raise Exception("No valid frames read from background video.")

    bg_len = len(bg_frames)

    while True:
        ret_fg, fg_frame = cap_fg.read()
        ret_alpha, alpha_frame = cap_alpha.read()

        if not ret_fg or not ret_alpha:
            break

        alpha = alpha_frame[:, :, 0:1] / 255.0
        alpha = np.repeat(alpha, 3, axis=2)

        bg_frame = bg_frames[frame_count % bg_len]
        composite = (alpha * fg_frame + (1 - alpha) * bg_frame).astype(np.uint8)

        out.write(composite)
        frame_count += 1

        if frame_count % 10 == 0:
            print(f"Processed {frame_count} frames...")

    cap_fg.release()
    cap_alpha.release()
    cap_bg.release()
    out.release()



    cloudinary_compatible_path = output_path
    command = [
        "ffmpeg",
        "-y",  # Overwrite if exists
        "-i", temp_output_path,
        "-c:v", "libx264",
        "-pix_fmt", "yuv420p",
        "-movflags", "+faststart",
        "-preset", "veryfast",
        "-crf", "23",
        cloudinary_compatible_path
    ]

    print("Encoding for Cloudinary:", " ".join(command))
    process = subprocess.run(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)

    if process.returncode != 0:
        raise Exception("FFmpeg failed:\n" + process.stderr)


    return cloudinary_compatible_path


async def download_file(url, filename):
    async with aiohttp.ClientSession() as session:
        async with session.get(url) as response:
            if response.status != 200:
                raise Exception(f"Failed to download {url}")
            with open(filename, "wb") as f:
                while True:
                    chunk = await response.content.read(1024)
                    if not chunk:
                        break
                    f.write(chunk)
                f.flush()
                os.fsync(f.fileno())

@app.post("/bgchange")
async def change_background_from_url(
    video_url: str = Query(..., description="URL to input video"),
    bg_number: int = Query(..., description="Number referring to background video")
):
    os.makedirs("temp", exist_ok=True)

    input_path = "temp/input_video.mp4"
    bg_path = f"/content/{bg_number}.mp4"
    fg_path = "temp/fg.mp4"
    alpha_path = "temp/pha.mp4"
    output_path = "temp/composited_output.mp4"

    await download_file(video_url, input_path)

    time.sleep(1)

    run_rvm(input_path,fg_path,alpha_path)

    if not os.path.exists(fg_path) or not os.path.exists(alpha_path):
      raise Exception("Foreground or alpha video not generated. RVM may have failed.")


    composite_video(fg_path, alpha_path, bg_path, output_path)
    print("Returning final video file to client:", output_path)
    return FileResponse(output_path, media_type="video/mp4", filename="cloudinary_output.mp4")


In [5]:
nest_asyncio.apply()

import threading
from threading import Thread
import time

def run():
    uvicorn.run(app, host="0.0.0.0", port=8000)

thread = threading.Thread(target=run)
thread.start()
thread.join(timeout=1.0)
ngrok.kill()
public_url = ngrok.connect(8000)
print("Public FastAPI URL:", public_url)

INFO:     Started server process [919]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


🚀 Public FastAPI URL: NgrokTunnel: "https://2f9c40641780.ngrok-free.app" -> "http://localhost:8000"
